# Librerías

In [ ]:
#!pip install pysqlite3
#!pip install scikit-lear
!pip install numpy==1.23.5
!pip install scikit-surprise

In [ ]:
# Manejo de Datos
import numpy as np
import pandas as pd
import sqlite3 as sql
import os
import sys
import datetime

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Estadísticas y Pruebas
import scipy.stats as stats
from scipy.stats import gaussian_kde

# Procesamiento de Datos
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Modelado y Algoritmos
from sklearn import neighbors
import joblib
#from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

# Interactividad
from ipywidgets import interact

# Google Colab
from google.colab import drive
from google.colab import files

# Otros
from collections import Counter

#Librerias de la API fe Gemini
import google.generativeai as genai #Para API de Gemini
import requests # Se usa para hacer solicitudes HTTP. Permite enviar peticiones POST a las APIs para traducción y generación de texto

In [ ]:
from surprise import Reader, Dataset #Libreria para modelos de recomendación

# Conectar con google drive


In [ ]:
drive.flush_and_unmount() #Linea en caso de tener que desconectar el drive por algún tipo de falla

Drive not mounted, so nothing to flush and unmount.


In [ ]:
#drive.flush_and_unmount()  #Linea en caso de tener que desconectar el drive por algún tipo de falla
drive.mount('/content/drive') #Linea para conectar al drive

Mounted at /content/drive


In [ ]:
path="/content/drive/MyDrive/analitica 3/sistemas_recomendacion" ### ruta del repositorio en drive
os.chdir(path) ### volver la carpeta del repositorio directorio de trabajo
sys.path.append(path) ### agregarla al path, poder leer archivos de funciones propios como paquetes

In [ ]:
import a_funciones as fn #Importar el documento de funciones para hacer uso de estas

# Base de datos

In [ ]:
conn = sql.connect('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/data/db_movies3') #Crear la conexión con la base de datos
cur = conn.cursor() #Creacion del cursos para realizar consultas dentro del mismo SQL

In [ ]:
# Creación de cursor para  ejecutar consultas en la base de datos
# Visualizar las tablas contenidas en la base de datos
cur.execute("SELECT name FROM sqlite_master where type='table'")
cur.fetchall()

[('movies_final',),
 ('ratings_final',),
 ('df_final',),
 ('df_terminado',),
 ('df_catalogo',),
 ('reco',)]

In [ ]:
def recomendar(uid=5, iid='5'):
